In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/nanoGPT-master/nanoGPT-master/


/content/drive/MyDrive/nanoGPT-master/nanoGPT-master


In [ ]:
ls

assets/          loss-graphs/                 README.md
bench.py         losstrain.py                 sample.py
config/          model.py                     scaling_laws.ipynb
configurator.py  out-shakespeare-char/        train.py
data/            out-shakespeare-char-fresh/  transformer_sizing.ipynb
LICENSE          __pycache__/


In [ ]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python data/shakespeare_char/prepare.py


length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [ ]:
!python losstrain.py config/train_shakespeare_char.py \
  --device=cuda --compile=False \
  --max_iters=500 --log_interval=1 --eval_iters=20


Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

In [ ]:
!python sample.py --out_dir=out-shakespeare-char --device=cuda


Overriding: out_dir = out-shakespeare-char
Overriding: device = cuda
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...


All before with and in the servainst beaut of and rud my called
My Lord, and will her back that ane away, and hand a way.

QUEEN ELIZABETH:
I am this nown that mise, sir, in the Herily,
Stengther flow him servised like to ear.

AUTOLYCUS:
Now up the speak you lord.
I came any and with the self that
To Windon have I besteed my curt a cent in honour.

ANGELO:
The womater fare so;
And he must with all of my said.

COMINIUS:
He could still is a honour,
And he matting must are no brother,
And for hi
---------------

Men pardon, marry how I was my mast,
Been so me noble the men.

CLAUDIO:
He dissenature your house.

LUCIO:
Master man, more.

Rome:
My lord.

LUCIO:
Ay, look to much the tory sight of their so now his raye,
And say, come, and suchild to loud to entreman of the banish a
were but with soul.

CATESBY:
And so welc no lord and the co

In [ ]:
!python losstrain.py config/train_shakespeare_char.py \
  --device=cuda --compile=False \
  --max_iters=1000 --log_interval=10 --eval_iters=20 \
  --init_from=resume --out_dir=out-shakespeare-char


Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

In [ ]:
!python sample.py --out_dir=out-shakespeare-char --device=cuda


Overriding: out_dir = out-shakespeare-char
Overriding: device = cuda
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...


KING RICHARD III:
The law bloody soul of Buckingham, the called
My Lord Hastings, he that did him enfrice my faces
Of my soul absenged forth him to the world.
Which, son mine are this own so deep?

GLOUCESTER:
How lord doth his humble child? therefore he tyrant?

GLOUCESTER:
Ay, good thy world that more counterfeitions to them?

BUCKINGHAM:
Ay, so more for the king thrust for their honour throness,
Intending their wits.

HENRY BOLINGBROKE:
Sadness, I am the hung courtears.

HENRY BOLINGBROKE:
O
---------------

Men pardon, I can infector with him,
Being one on my heart's lady. Your country's name,
Your house countenants the might of England,
My work.

LADY ANNE:
How! that dissolute of France?
You must not hear me speak him, and looks his heart,
Which in her man I was have with her brother me.

DUKE VINCENTIO:
What lack to his heart and 